In [0]:
dbutils.widgets.text("Environment","mycatalog","set the current environment")
dbutils.widgets.text('RunType',"once","set once to run as a batch")
dbutils.widgets.text("processingtime","5 seconds","set the microbatch interval")

In [0]:
env= dbutils.widgets.get("Environment")
once = True if dbutils.widgets.get("RunType")=="once" else False
processingTime = dbutils.widgets.get("processingtime")
if once:
    print ('starting in batch mode')
else:
    print('starting in stream mode {processingTime} microbatch.')

In [0]:
# Performance tuning configutration 

spark.conf.set ('spark.sql.shuffle.partitions',sc.defaultParallelism)
spark.conf.set ('spark.databricks.delta.optimizeWrite.enabled',True)
spark.conf.set('spark.databricks.delta.autoCompact.enabled',True)
spark.conf.set('spark.sql.streaming.stateStore.providerClass', 'com.databricks.sql.streaming.statestore.RocksDBStateStoreProvider')

In [0]:
%run ./02-setup

In [0]:
%run ./03-history-loader

In [0]:
sh = SetupHelper(env)
hl = HistoryLoader(env)

In [0]:
setup_required  = spark.sql(f'show databases in {sh.catalog}).filter(f"databaseName == '{sh.db_name}' ").count() ! =1
if setup_required:
    sh.setup()
    sh.validate()
    hl.load_history()   
    hl.validate()
else:
    spark.sql(f"use {sh.catalog}.{sh.db_name}")

In [0]:
%run ./04-bronze-layer

In [0]:
%run ./05-silver-layer

In [0]:
%run ./06-gold-layer

In [0]:
bl= bronze(env)
sl = silver(env)
gl = Gold(env)

In [0]:
bl.consume(once,processing_time)

In [0]:
sl.upsert(once,processing_time)

In [0]:
gl.upsert(once,processing_time)